In [14]:
import numpy as np

In [39]:
import pandas as pd
from pandas import DataFrame
def import_data(filename):
    data = pd.read_csv(filename, sep=',')
    return data

# print(df.head())
df = import_data('../../datasets_clean/demographics.csv')
df['zip'] = df['zip'].apply(lambda x: '{0:0>5}'.format(x))
# df.columns
# df


In [38]:
import folium
import json
import numpy as np
def get_center_latlong(df):
    #center lat long of boston
    return (42.361145, -71.057083)
def plot_demographic_heatmap(df,col, output_path):
    # this line just to force our dataset zipcodes from numbers to strings so they match our geojson file
    df['zip'] = df['zip'].astype('str')


    # load my geojson file which contains my Polygons
    boundary_file = '../../datasets_clean/Boston_ZIP_Codes.geojson'
    with open(boundary_file, 'r') as f:
        zipcode_boundary = json.load(f)


    # Grab my map center again using our created fucntion (same as before)
    center = get_center_latlong(df)


    # Initialize Folium Map again (same as before)
    m = folium.Map(location=center, 
                   zoom_start=10)


    # Use the groupby method to 
    zipcode_data = df.groupby('zip').aggregate(np.mean)
    zipcode_data.reset_index(inplace = True)


    # Create choropleth map  
    folium.Choropleth(
        geo_data=zipcode_boundary,
        name='choropleth',
        data=zipcode_data,
        columns=['zip', col],
        key_on='feature.properties.ZIP5',
        fill_color='Spectral',
        fill_opacity=0.6,
        nan_fill_opacity=0,
        line_opacity=1,
        legend_name='Avg Density'
    ).add_to(m)


    m.save(output_path)

In [40]:
print(df.columns)

Index(['Unnamed: 0', 'zip', 'po_name', 'pop2010', 'pop2013', 'white', 'black',
       'ameri_es', 'asian', 'hawn_pi', 'hispanic', 'other', 'mult_race',
       'males', 'females', 'age_under5', 'age_5_9', 'age_10_14', 'age_15_19',
       'age_20_24', 'age_25_34', 'age_35_44', 'age_45_54', 'age_55_64',
       'age_65_74', 'age_75_84', 'age_85_up', 'med_age', 'med_age_m',
       'med_age_f', 'households', 'ave_hh_sz', 'hsehld_1_m', 'hsehld_1_f',
       'marhh_chd', 'marhh_no_c', 'mhh_child', 'fhh_child', 'families',
       'ave_fam_sz'],
      dtype='object')


In [ ]:

plot_demographic_heatmap(df,'hispanic','maps/hispanic.html')
plot_demographic_heatmap(df,'white','maps/white.html')
plot_demographic_heatmap(df,'hispanic','maps/hispanic.html')
plot_demographic_heatmap(df,'hispanic','maps/hispanic.html')
plot_demographic_heatmap(df,'hispanic','maps/hispanic.html')
